# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [130]:
# Details Student 1:
print("Name:Roy Twizer")
print("ID:203926084")



Name:Roy Twizer
ID:203926084


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [131]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [132]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [113]:
# word net installation:

# unmark if you want to use and need to install
 
!pip install wn
!python -m wn download omw-he:1.4

Defaulting to user installation because normal site-packages is not writeable



Cached file found: C:\Users\יוכבד\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\2C43~1\AppData\Local\Temp\tmp4fpcstz5\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [114]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [115]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer


Defaulting to user installation because normal site-packages is not writeable


In [116]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [117]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [118]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [119]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [135]:
def preprocess_text(df: pd.DataFrame):
    # Removing non Hebrew characters
    df['story'] = df['story'].str.replace(r'[^\u0590-\u05fe\s]+', '', regex=True)

    # STEMMING:
    prefixes_to_remove = ['כ', 'ו', 'ב', 'ש', 'ה', 'ל', 'כש', 'לכש', 'וש', 'וכש']
    for prefix in prefixes_to_remove:
        df['story'] = df['story'].apply(lambda x: re.sub(fr'\b{prefix}\b', '', x))
    
    suffixes_to_remove = ['תי', 'נו', 'ים', 'ו']
    for suffix in suffixes_to_remove:
        df['story'] = df['story'].apply(lambda x: re.sub(fr'\b{suffix}\b', '', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'ות\b', 'ה', x))
    
    words_to_fix = {'מ': 'ם', 'נ': 'ן', 'פ': 'ף', 'כ': 'ך', 'צ': 'ץ'}
    for original, replacement in words_to_fix.items():
        df['story'] = df['story'].apply(lambda x: re.sub(fr'\b{original}\b', replacement, x))
    
    # Removing words that have less than 3 letters - many of them are stop words
    df['story'] = df['story'].str.replace(r'\b\w{1,2}\b', '', regex=True)
    
    # Removing multiple spaces
    df['story'] = df['story'].apply(lambda x: re.sub(r'\s+', ' ', x))



In [121]:
preprocess_text(df_train)
preprocess_text(df_test)

In [122]:
df_train['story'][2]


'מאז שהתחילו הלימודים חלומו סטודנט הפנגויה מחפשים חברים וקובעים קבוצה היה להכיר אנשים חדשים במכללה במיוחד שלא מכירים אחד מראש אבל אחרי מאמצים הצלחתי למצוא לעצמי חברה אנשים נחמדים להיה איתם וכבר בחודש הראשון התואר ידעתי אני נוסעת חברה חדשה שפגשתי בכיתה שלי ורצינו ללכת ביחד כולנו באותו ראש עובדה חלק רווקה וחלק בזוגיה אבל שהכי רצינו לעשה ללכת לחגוג ישר קבענו ביחד אפילו החדרים שלנו סגרנו כשהייתי בחול אצל לקוח החברה שלי ככה שלא היה אכפת לפנה לזה זמן כדי שנוכל ללכת ולהנה ואז התחילה הקורונה ושיהשה התוכניה יצא שבמשך תקופה מאוד ארוכה דחו הקורונה בלי הפסקה מספר פעמים שבמקום מאי אותה שנה הפנגויה היתה בספטמבר שנתיים אחרי וגם היה שינויים במסיבה ובתוכניה הפנגויה כלומר פנגויה רגילה אבל אחרי שנתיים לאף אחד אכפת וכל אחת מהבנה היתה בסטטוס אחר וכך יצא שבמקום בנה נשארנו אבל לפחה החברה הכי טובה במשך תקופה מאוד ארוכה תיכננו נקנה ומה נלבש ולאן נצא אפילו קבענו מקום למסעדה שנקראת המחבוא אדי אנשים כמה חודשים מראש ואז הגיע השבוע לפני הפנגויה ההתרגשה בשיא אבל אני בהרגשה שיא בהתחלה היה צינון ואחרי היה שיעול וחום ו

In [123]:
def get_tokens(text):
    return text.split(' ')

In [124]:
def get_tokens(text):
    return text.split(' ')

def vectorizer(df_train, df_test):
    # max_df=400 removes many of the stop words
    # min_df=4 removes some typos and rare words
    vect = CountVectorizer(tokenizer=get_tokens, ngram_range=(1, 3), min_df=5, max_df=400, max_features=10000)

    combined_df = pd.concat([df_train, df_test], axis=0)

    X = vect.fit_transform(combined_df['story'].to_list())
    y = df_train[df_train.columns[-1]].replace({'m': 0, 'f': 1})  # mapping 'm' to 0 and 'f' to 1

    feature_names = vect.get_feature_names_out()
    display(pd.DataFrame(X.toarray(), columns=feature_names).iloc[:, 100:121])

    train_X = X[:len(df_train)]
    test_X = X[len(df_train):]

    return train_X, test_X, y

In [125]:
X_train, X_test, y_train = vectorizer(df_train, df_test)

,אוהלים,אוויר,אוויר היה,אווירה,אווירה טובה,אוטו,אוטובוס,אוטובוסים,אוטומטי,אוי,אוכל,אוכל ומים,אוכל טעים,אוכל להגיע,אוכלים,אולי,אולי אפילו,אולי יהיה,אולם,אולמה,אומנה
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1072,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
1073,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [126]:
# cross validation
from sklearn.metrics import make_scorer

In [127]:
def cross_validation(clf, X, y):
    # 10 folds, average f1 score by default 
    scores = cross_val_score(clf, X, y, scoring=make_scorer(f1_score, average='macro'), cv=10)
    return scores

In [128]:
# Initialize an empty list to hold prediction results
predicted_results = []

In [137]:
# Data preparation
# Assuming you have your df_train and df_test DataFrames
X_train, X_test, y_train = vectorizer(df_train, df_test)

# List of classifiers to compare
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=42),
    MultinomialNB(),
    GaussianNB()
]
# The parameter grid for each classifier
param_grids = [
    {'n_neighbors': [3, 5, 7]},
    {'max_depth': [2, 4, 6, 8, 10]},
    {},
    {}
]
scoring_metric = make_scorer(f1_score, average='macro')


,אוהלים,אוויר,אוויר היה,אווירה,אווירה טובה,אוטו,אוטובוס,אוטובוסים,אוטומטי,אוי,אוכל,אוכל ומים,אוכל טעים,אוכל להגיע,אוכלים,אולי,אולי אפילו,אולי יהיה,אולם,אולמה,אומנה
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1072,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
1073,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [138]:
# Initialize lists to hold captured output for each classifier
output_classifier_names = []
output_parameters = []
output_scores = []

for clf, param_grid in zip(classifiers, param_grids):
    grid_search = GridSearchCV(clf, param_grid, cv=10, scoring=scoring_metric, n_jobs=-1)
    grid_search.fit(X_train.toarray(), y_train)
    
    # Capture output for each classifier
    output_classifier_names.append(clf.__class__.__name__)
    output_parameters.append(grid_search.best_params_)
    output_scores.append(grid_search.best_score_)
    scoring_metric = make_scorer(f1_score, average='macro')
    print('Best parameters for {}: {}'.format(clf.__class__.__name__, grid_search.best_params_))
    print('Best {} score for {}: {:.2f}'.format(scoring_metric, clf.__class__.__name__, grid_search.best_score_))

# Create a DataFrame from the captured output
output_df = pd.DataFrame({
    'Classifier': output_classifier_names,
    'Best Parameters': output_parameters,
    'Best Score': output_scores
})



GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for KNeighborsClassifier: {'n_neighbors': 3}
Best make_scorer(f1_score, average=macro) score for KNeighborsClassifier: 0.47


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             n_jobs=-1, param_grid={'max_depth': [2, 4, 6, 8, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for DecisionTreeClassifier: {'max_depth': 8}
Best make_scorer(f1_score, average=macro) score for DecisionTreeClassifier: 0.60


GridSearchCV(cv=10, estimator=MultinomialNB(), n_jobs=-1, param_grid={},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for MultinomialNB: {}
Best make_scorer(f1_score, average=macro) score for MultinomialNB: 0.72


GridSearchCV(cv=10, estimator=GaussianNB(), n_jobs=-1, param_grid={},
             scoring=make_scorer(f1_score, average=macro))

Best parameters for GaussianNB: {}
Best make_scorer(f1_score, average=macro) score for GaussianNB: 0.48


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [134]:
# Save the DataFrame to a CSV file
output_df.to_csv('classification_results.csv', index=False)

print("Classifier results saved to 'classification_results.csv'")

Classifier results saved to 'classification_results.csv'
